#### Observe that we can get news of a day with the link format:
http://news.ltn.com.tw/list/newspaper/politics/20181231
#### We need a list of date string in the date range we want.

In [25]:
#-*- coding:utf-8 -*-
from datetime import datetime, timedelta

start_date = "2019-02-28" # set start Date
stop_date = "2019-02-28" # set stop Date

start = datetime.strptime(start_date, "%Y-%m-%d")
stop = datetime.strptime(stop_date, "%Y-%m-%d")

dates = list()
while start <= stop:
    dates.append(start.strftime('%Y/%m/%d'))
    start = start + timedelta(days=1)


#### Now write a function to parse the HTML response, return the data we want(title, content, ...etc).

In [26]:
import requests
from bs4 import BeautifulSoup as bs

In [27]:
def process_document(document, date):
    newsBody = document.find(id='ranking_body')
    newsList = newsBody.select('table > tr')
    link = ''
    tag = ''
    title = ''

    data = list()
    
    for tr in newsList:
        td = tr.select('td')
        for element in td:
            if element.select_one('a') != None:
                # get link
                link = 'https://udn.com' + element.select_one('a')['href']
                # get title
                title = element.string
                # get content
                res = requests.get(link)
                doc = bs(res.text, 'lxml')
                body = doc.find(id='story_body_content')
                content = ""
                for ele in body.select('p'):
                    if ele.string != None:
                        content += ele.string
            elif ((len(element.get('class')) == 1) and (element.get('class')[0] == 'only_web')):
                # get tag
                tag = element.string
            else:
                continue
        if link != '':
            data.append({
            'date' : date,
            'title': title,
            'link' : link,
            'content' : content, 
            'tags' : tag
            })
        
    return data

#### Crawl over the news on the site, store the data in variable "all_data" .

In [28]:
cnt = 0
all_data = list()
for date in dates:
    print('start crawling :', date)
    res = requests.get('https://udn.com/news/archive/0/0/' + date)
    doc = bs(res.text, 'lxml')
    data = process_document(doc, date)
    all_data += data

('start crawling :', '2019/02/28')


#### Check the result

In [29]:
all_data[0:5]

[{'content': u'16:48\u5ddd\u666e\u5728\u4ed6\u4e0b\u69bb\u7684\u842c\u8c6a\u9152\u5e97\u8209\u884c\u7684\u8a18\u8005\u6703\uff0c\u6b77\u6642\u7d0440\u5206\u9418\uff0c\u4ed6\u5728\u7d50\u675f\u524d\u8aaa\uff1a\u300c\u6211\u5373\u5c07\u642d\u6a5f\u8fd4\u7a0b\uff0c\u56de\u5230\u83ef\u76db\u9813DC\u90a3\u500b\u597d\u5730\u65b9\u3002\u300dBBC\u5831\u5c0e\uff0c\u5ddd\u666e\u986f\u7136\u4e0d\u60f3\u5728\u6cb3\u5167\u591a\u5f85\u4e00\u5206\u9418\uff0c16:40\u62b5\u9054\u6a5f\u5834\uff0c\u8ddf\u6771\u9053\u4e3b\u8d8a\u5357\u5b98\u54e1\u9053\u5225\uff0c\u7a7a\u8ecd\u4e00\u865f\u572816:48\u8d77\u98db\u300215:59\u5ddd\u666e\u8ac7\u53ca\u548c\u91d1\u6b63\u6069\u95dc\u4fc2\uff0c\u8868\u793a\u5169\u4eba\u7684\u95dc\u4fc2\u300c\u975e\u5e38\u6eab\u6696\u300d\uff0c\u4e26\u5805\u7a31\u5cf0\u6703\u4e26\u975e\u5728\u722d\u5435\u7684\u7dca\u7e43\u72c0\u6cc1\u4e0b\u7d50\u675f\uff0c\u800c\u662f\u300c\u4e00\u6bb5\u975e\u5e38\u53cb\u597d\u7684\u6563\u6b65\u300d\u3002\u4ed6\u56de\u7b54\u5a92\u9ad4\u63d0\u554f\u8a